# Image classification with fast.ai
In this notebook you will learn how to make in a few lines of code a model that distinguises cats from dogs in pictures, using the [fast.ai](https://docs.fast.ai/) library.

To train a deep learning model, a GPU runtime is needed. With datalayer, you can switch your kernel to a remote kernel with GPU using the kernel picker.

## Disclaimer
This is widely based on the [fastbook](https://github.com/fastai/fastbook) created by [Jeremy Howard](https://github.com/jph00) and [Sylvain Grugger](https://github.com/sgugger). 


## Setup

In [ ]:
!pip install fastai

In [2]:
# GPU check
!nvidia-smi

Thu Jul 11 14:52:09 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100S-PCIE-32GB          On  |   00000000:00:05.0 Off |                    0 |
| N/A   39C    P0             25W /  250W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Dataset

A dataset called the [Oxford-IIIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/) that contains 7,349 images of cats and dogs from 37 different breeds will be downloaded from the fast.ai datasets collection and will then be extracted.

In [3]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'  # downloads the data

def is_cat(x): return x[0].isupper()  # label function (if uppercase it is a cat, otherwise it is a dog)

dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))  # data loader

We can then check if everything looks okay with the `show_batch` method (`True` is for cat, `False` is for dog):

In [ ]:
dls.show_batch()

## Model training

A *pretrained model* that has already been trained on 1.3 million images, using a competition-winning model will be downloaded from the internet.

The pretrained model will be *fine-tuned* using the latest advances in transfer learning, to create a model that is specially customized for recognizing dogs and cats. It will run two epochs, one only on 

The fine-tuning needs to be run on GPU.

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate, cbs=ShowGraphCallback())  # our CNN model
learn.fine_tune(2)  # training of the model

## Predict on new data

In [6]:
from IPython.display import Image, HTML, display

In [ ]:
! curl https://raw.githubusercontent.com/fastai/fastbook/master/images/chapter1_cat_example.jpg -o /tmp/cat.jpg
! ls /tmp/cat.jpg
Image("/tmp/cat.jpg")

In [12]:
pred, _, probs = learn.predict("/tmp/cat.jpg")
display(HTML(f"<h1>Is this a cat?: {pred}.</h1>"))
print(f"Probability it's a cat: {probs[1].item():.6f}")

Probability it's a cat: 1.000000


In [ ]:
! curl https://cdn.mos.cms.futurecdn.net/ASHH5bDmsp6wnK6mEfZdcU.jpg -o /tmp/dog.jpg
! ls /tmp/dog.jpg
Image("/tmp/dog.jpg")

In [18]:
pred, _, probs = learn.predict("/tmp/dog.jpg")
display(HTML(f"<h1>Is this a cat?: {pred}.</h1>"))
print(f"Probability it's a cat: {probs[1].item():.6f}")

Probability it's a cat: 0.000102


## Saving our model

In [16]:
learn.export('export.pkl')